In [13]:
import pandas as pd
sql_records_df = pd.read_csv("pandas_df.csv")

In [14]:
sql_records_df

,full_url,repeat_count
0,/details.php?pr\tod=5&type=2,1
1,/details.php?prod=5&type=2&OUOT=7916%20AND%201...,2
2,/details.php?prod=5&type=2,3
3,/details.php?prod=3196&type=2,4
4,/details.php?prod=5.%28%2C%2C%2C%27%28%22%28%2...,5
...,...,...
3963,/details.php?prod=5&type=-7429%27%20IN%20BOOLE...,3964
3964,/details.php?prod=5&type=-1883%27%20IN%20BOOLE...,3965
3965,/details.php?prod=5&type=-5655%27%20IN%20BOOLE...,3966
3966,/details.php?prod=5&type=-6048%27%20IN%20BOOLE...,3967


In [86]:
from urllib.parse import unquote, urlparse, parse_qs

def url_decoder(full_url):
    full_url = unquote(full_url)
    return full_url

sql_records_df['parsed_url'] = sql_records_df['full_url'].apply(url_decoder)

In [17]:
sql_records_df = sql_records_df.reindex(columns=['full_url','parsed_url','repeat_count'])

In [19]:
sql_records_df

,full_url,parsed_url,repeat_count
0,/details.php?pr\tod=5&type=2,/details.php?pr\tod=5&type=2,1
1,/details.php?prod=5&type=2&OUOT=7916%20AND%201...,/details.php?prod=5&type=2&OUOT=7916 AND 1=1 U...,2
2,/details.php?prod=5&type=2,/details.php?prod=5&type=2,3
3,/details.php?prod=3196&type=2,/details.php?prod=3196&type=2,4
4,/details.php?prod=5.%28%2C%2C%2C%27%28%22%28%2...,"/details.php?prod=5.(,,,'(""(,&type=2",5
...,...,...,...
3963,/details.php?prod=5&type=-7429%27%20IN%20BOOLE...,/details.php?prod=5&type=-7429' IN BOOLEAN MOD...,3964
3964,/details.php?prod=5&type=-1883%27%20IN%20BOOLE...,/details.php?prod=5&type=-1883' IN BOOLEAN MOD...,3965
3965,/details.php?prod=5&type=-5655%27%20IN%20BOOLE...,/details.php?prod=5&type=-5655' IN BOOLEAN MOD...,3966
3966,/details.php?prod=5&type=-6048%27%20IN%20BOOLE...,/details.php?prod=5&type=-6048' IN BOOLEAN MOD...,3967


In [77]:
import requests
from bs4 import BeautifulSoup

# scraping keywordow
sql_keywords = []

url = "https://www.w3schools.com/sql/sql_ref_keywords.asp"
resp = requests.get(url)
print(resp.status_code)

soup = BeautifulSoup(resp.text, 'html.parser')
for keyword in soup.findAll('td'):
    keyword = keyword.find('a')
    if(keyword!=None):
        sql_keywords.append(keyword.text)

200


In [81]:
import re

sql_keywords_counter = []

for record in sql_records_df["parsed_url"]:
    tmp = 0
    for keyword in sql_keywords:
        if(re.search(keyword,record)):
            tmp = tmp +1
    sql_keywords_counter.append(tmp)

sql_records_df["sql_keyword"] = sql_keywords_counter

In [82]:
sql_records_df

,full_url,parsed_url,repeat_count,sql_keyword
0,/details.php?pr\tod=5&type=2,/details.php?pr\tod=5&type=2,1,0
1,/details.php?prod=5&type=2&OUOT=7916%20AND%201...,/details.php?prod=5&type=2&OUOT=7916 AND 1=1 U...,2,8
2,/details.php?prod=5&type=2,/details.php?prod=5&type=2,3,0
3,/details.php?prod=3196&type=2,/details.php?prod=3196&type=2,4,0
4,/details.php?prod=5.%28%2C%2C%2C%27%28%22%28%2...,"/details.php?prod=5.(,,,'(""(,&type=2",5,0
...,...,...,...,...
3963,/details.php?prod=5&type=-7429%27%20IN%20BOOLE...,/details.php?prod=5&type=-7429' IN BOOLEAN MOD...,3964,5
3964,/details.php?prod=5&type=-1883%27%20IN%20BOOLE...,/details.php?prod=5&type=-1883' IN BOOLEAN MOD...,3965,5
3965,/details.php?prod=5&type=-5655%27%20IN%20BOOLE...,/details.php?prod=5&type=-5655' IN BOOLEAN MOD...,3966,5
3966,/details.php?prod=5&type=-6048%27%20IN%20BOOLE...,/details.php?prod=5&type=-6048' IN BOOLEAN MOD...,3967,5


In [157]:
params_length = []

for record in sql_records_df["parsed_url"]:
    tmp = 0
    params = []

    for i in parse_qs(record).values():
       # print(str(i))
       # print(re.search(r"\['(\b.*)'\]",str(i)).group(1))
       # params.append(re.search(r"\['(\b.*)'\]",str(i)).group(1))
        params.append(str(i)[2:-2])
    
    for param in params:
        tmp = tmp + len((param))

    params_length.append(tmp)
  
sql_records_df["params_length"] = params_length

In [158]:
sql_records_df

,full_url,parsed_url,repeat_count,sql_keyword,params_length
0,/details.php?pr\tod=5&type=2,/details.php?pr\tod=5&type=2,1,0,2
1,/details.php?prod=5&type=2&OUOT=7916%20AND%201...,/details.php?prod=5&type=2&OUOT=7916 AND 1=1 U...,2,8,178
2,/details.php?prod=5&type=2,/details.php?prod=5&type=2,3,0,2
3,/details.php?prod=3196&type=2,/details.php?prod=3196&type=2,4,0,5
4,/details.php?prod=5.%28%2C%2C%2C%27%28%22%28%2...,"/details.php?prod=5.(,,,'(""(,&type=2",5,0,13
...,...,...,...,...,...
3963,/details.php?prod=5&type=-7429%27%20IN%20BOOLE...,/details.php?prod=5&type=-7429' IN BOOLEAN MOD...,3964,5,77
3964,/details.php?prod=5&type=-1883%27%20IN%20BOOLE...,/details.php?prod=5&type=-1883' IN BOOLEAN MOD...,3965,5,82
3965,/details.php?prod=5&type=-5655%27%20IN%20BOOLE...,/details.php?prod=5&type=-5655' IN BOOLEAN MOD...,3966,5,87
3966,/details.php?prod=5&type=-6048%27%20IN%20BOOLE...,/details.php?prod=5&type=-6048' IN BOOLEAN MOD...,3967,5,92


# Metryki wykorzystane w analizie:

 1. znaki specialne (' ; : } -- =) - stosunek znakw normalnych i tych innych
 2. zrobic algorytm liczacy ilosc zapytan na minute

- ogarnąć żeby filtr nie był case sensitive (jes tflaga w SNORT)
- zrobić stosunek znaków wszystkich do specialnych (zdefiniować tą listę znaków specialnych) 
- wykryć wykorzystanie komentarza 
- wykryć zamknięcie parametru (nawiasy)
- analiza długości parametrów (=<*>&)

czy dla każdego wyrażenie robimy: kolumne binarną/robimy counter wyrażeń specialnych/ counter poszczególnych słów kluczowych?

## Lista skanerów SQLinjection:
- SQLMap – Automatic SQL injection and database 
- takeover tool
- jSQL Injection – Java tool for automatic SQL
- database injection
- BBQSQL – A blind SQL -injection exploitation tool
- NoSQLMap – automated NoSQL database pwnage
- Whitewidow – SQL vulnerability scanner
- DSSS – Damn small SQLi scanner
- Explo – Human and machine-readable web
- vulnerability testing format
- Blind-SQL-Bitshifting – Blind SQL-injection via
- bit-shifting
- Leviathan – Wide range mass audit toolkit
- Blisqy – Exploit Time-based blind-SQL-injection in
- HTTP-headers (MySQL/MariaDB)

## Linki do literatury


##



## Bot ktory generuje dobry ruch


<obrazkeprzejsc dla mszyny stanow moora/milliego>
- bot maskuje sie i symuluje kilku uzytkownikow na raz

## Analiza danych

- korelacja itp (metryki ktore analizujemy)

